# Disney Dataset Creation

Webscraping solution using beautifulsoup
Following along Keith Galli's video: https://www.youtube.com/watch?v=Ewgy-G9cmbg

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import json
import re
from datetime import datetime
import pickle
import os
import urllib

In [2]:
# getting the page
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# creating the soup
soup = bs(r.content)

# making it readable
contents = soup.prettify()

In [3]:
# getting only the infobox with the main info
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")


In [4]:
movie_info = {}


def get_content(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find('th').get_text()
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content(row.find('td'))
        movie_info[content_key] = content_value
            

In [5]:
# getting the page
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# creating the soup
soup = bs(r.content)

# making it readable
contents = soup.prettify()

In [6]:
def get_content(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")
    
def clean_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose()
    for tag in soup.find_all("span"):
        tag.decompose()
        
def get_info_box(url):
    r = requests.get(url)
    soup = bs(r.content)
    
    clean_tags(soup)
    
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text()
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content(row.find('td'))
                movie_info[content_key] = content_value
                
    return movie_info

In [7]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select('.wikitable.sortable i a')
base_path = 'https://en.wikipedia.org'
movie_info_list = []

for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
        
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

0


KeyboardInterrupt: 

In [8]:
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [9]:
def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [50]:
save_data("disney_data_cleaned.json", movie_info_list)

### Data Cleaning 

In [10]:
movie_info_list = load_data("disney_data_cleaned.json")

#### List of subtasks

1. using python datetime 
2. ~~convert running type and money to integer~~
3. ~~remove references [1]~~
4. ~~standardize data~~
5. ~~some 'starring' are not in a list~~
6. ~~look at what is going on at the error ones~~


In [ ]:
# Clean up references (remove [1], [2])

In [11]:
# convert running time into an integer
def minutes_to_integer(running_time):
    if running_time == "N/A":
        return None
    if isinstance(running_time, list):
        return int(running_time[0].split(" ")[0])
    else:
        return int(running_time.split(" ")[0])
        
for movie in movie_info_list:
    movie['Runnig time (int)'] = minutes_to_integer(movie.get('Running time', "N/A"))



In [12]:
print ([movie.get('Budget', 'N/A') for movie in movie_info_list])


['N/A', '$1.49 million', '$2.6 million', '$2.28 million', '$600,000', '$950,000', '$858,000', 'N/A', '$788,000', 'N/A', '$1.35 million', '$2.125 million', 'N/A', '$1.5 million', '$1.5 million', 'N/A', '$2.2 million', '$1,800,000', '$3 million', 'N/A', '$4 million', '$2 million', '$300,000', '$1.8 million', 'N/A', '$5 million', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$700,000', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$6 million', 'under $1 million or $1,250,000', 'N/A', '$2 million', 'N/A', 'N/A', '$2.5 million', 'N/A', 'N/A', '$4 million', '$3.6 million', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', '$4.4–6 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', '

In [13]:
# clean up budget & Box office

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

value_re = rf"\${number}"
word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"

'''
Possible values:
$600,000 -> 600000              ## value syntax
$12.2 million -> 12200000       ## word syntax (million, billion, etc)
$12-13 million -> 12000000      ## word syntax with a range
$16 to 20 million -> 16000000   ## word syntax with a different range
[12]
'''

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]


def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))

    word = re.search(amounts, string, flags=re.I).group().lower()
    total_amount = value * word_to_value(word)
    return total_amount

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

def money_conversion(money):
    if money == "N/A":
        return None
    
    if isinstance(money, list):
        money = money[0]

    value_syntax = re.search(value_re, money)
    word_syntax = re.search(word_re, money, flags=re.I)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())
    else:
        return None


In [14]:
for movie in movie_info_list:
    movie['Budget (float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', "N/A"))


In [15]:
for movie in movie_info_list:
    if movie.get('Release date') != None:
        print (movie.get('Release date'))
    elif movie.get('Release dates') != None:
        print (movie.get('Release dates'))
    else:
        print ("N/A")

['May 19, 1937']
['December 21, 1937 ( Carthay Circle Theatre )', 'February 4, 1938 (United States)']
['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)']
['November 13, 1940']
['June 27, 1941']
['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)']
['August 9, 1942 (World Premiere – London)', 'August 13, 1942 (Premiere – New York City)', 'August 21, 1942 (U.S.)']
['August 24, 1942 (World Premiere – Rio de Janeiro)', 'February 6, 1943 (U.S. Premiere – Boston)', 'February 19, 1943 (U.S.)']
['July 17, 1943']
['December 21, 1944 (Mexico City)', 'February 3, 1945 (US)']
['April 20, 1946 (New York City premiere)', 'August 15, 1946 (U.S.)']
['November 12, 1946 (Premiere: Atlanta, Georgia)', 'November 20, 1946', 'March 30, 1947 (Stanford Theatre, Palo Alto, California)']
['September 27, 1947']
May 27, 1948
['November 29, 1948 (Chicago, Illinois)', 'January 19, 1949 (Indianapolis, Indiana)']
['October 5, 1949']
['February 15, 1950 (Boston)', 'March 4, 1950 (Uni

In [16]:
# Transforming the date into a datetime python object
# types of date:
# July 24, 2009
# 20 July 2001

dates = [movie.get('Release date', movie.get('Release dates', "N/A")) for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
    
    if date == "N/A":
        return None
    
    date_str = clean_date(date)
    
    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str,fmt)
        except:
            pass
    return None
    

In [17]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', movie.get('Release dates', "N/A")))


In [18]:
# saving the data now using pickle to keep datetime format
# therefore, creating new save and load formats
def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [19]:
save_data_pickle("disney_movie_data_better_cleaned.pickle", movie_info_list)

In [20]:
movie_info_list = load_data_pickle('disney_movie_data_better_cleaned.pickle')

## Task #4 Attach IMDB/Rotten Totatoes/metascore scores

In [33]:
# using the OMDb API
def get_omdb_info(title):
    base_url = 'http://www.omdbapi.com/?'
    parameters = {'apikey': os.environ['OMDB_API_KEY'], 't': title} 
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None

In [39]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)
    

In [41]:
movie_info_list[-150]

{'title': 'Tales from Earthsea',
 'Japanese': '',
 'Hepburn': 'Gedo Senki',
 'Directed by': 'Gorō Miyazaki',
 'Screenplay by': ['Gorō Miyazaki', 'Keiko Niwa'],
 'Based on': ['Earthsea',
  'by',
  'Ursula K. Le Guin',
  "Shuna's Journey",
  'by',
  'Hayao Miyazaki'],
 'Produced by': ['Toshio Suzuki', 'Tomohiko Ishii'],
 'Starring': ['Bunta Sugawara', 'Junichi Okada', 'Aoi Teshima', 'Yūko Tanaka'],
 'Edited by': 'Takeshi Seyama',
 'Music by': 'Tamiya Terashima',
 'Production company': 'Studio Ghibli',
 'Distributed by': 'Toho',
 'Release date': ['July 29, 2006'],
 'Running time': '115 minutes',
 'Country': 'Japan',
 'Language': 'Japanese',
 'Budget': '$22 million',
 'Box office': '',
 'Runnig time (int)': 115,
 'Budget (float)': 22000000.0,
 'Box office (float)': None,
 'Release date (datetime)': datetime.datetime(2006, 7, 29, 0, 0),
 'imdb': '6.4',
 'metascore': '47',
 'rotten_tomatoes': None}

In [43]:
save_data_pickle('disney_movie_data_final.pickle', movie_info_list)

In [44]:
### Save data as json and csv
movie_info_copy = [movie.copy() for movie in movie_info_list]

In [45]:
for movie in movie_info_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [53]:
save_data('disney_movie_data_final.json', movie_info_copy)

In [49]:
import pandas as pd

In [50]:
df = pd.DataFrame(movie_info_list)

In [54]:
df.to_csv("disney_movie_data_final.csv")

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   title                    518 non-null    object        
 1   Production company       214 non-null    object        
 2   Distributed by           516 non-null    object        
 3   Release date             339 non-null    object        
 4   Running time             495 non-null    object        
 5   Country                  463 non-null    object        
 6   Language                 497 non-null    object        
 7   Box office               400 non-null    object        
 8   Runnig time (int)        495 non-null    float64       
 9   Budget (float)           307 non-null    float64       
 10  Box office (float)       389 non-null    float64       
 11  Release date (datetime)  500 non-null    datetime64[ns]
 12  imdb                     498 non-nul